In [209]:
import sqlite3
import csv
from contextlib import closing

In [213]:
class Product:
    catagory = "Product"
    def __init__(self,  iid,  name, price, qty):
        self.iid = iid
        self.name = name
        self.price = price
        self.qty = qty
    
class Book(Product):
    catagory = "BOOK"
    def __init__(self, catagory, iid,  name, author, price, qty):
        super().__init__(iid, name, price, qty)
        self.author = author
    
    def get_price(self, price, qty):
        total = self.price * self.qty
        return total

class Movie(Product):
    catagory = "MOVIE"
    def __init__(self, catagory, iid, name, genre, year, price, qty):
        super().__init__(iid, name, price, qty)
        self.__genre = genre
        self.__year = year
    
    def get_genre(self):
        return self.__genre
    
    def get_year(self):
        return self.__year
    
    def get_price(self, price, qty):
        total = self.price * self.qty
        return total

    
def add_product_menu(dbconn, products):
    """adds an product to the table"""
    with dbconn:
        ans = input("What would you like to add a movie or a book? ")
        ans = ans.upper()
        print(ans)
        if ans == 'MOVIE':
            iid = input("Item ID: ")
            name = input("Name: ")
            year = int(input("Year releaced: "))
            genre = input("Genre: ")
            price  = float(input("Price dont add '$' or ',': "))
            qty = int(input("Quantity: "))
            db.execute("INSERT INTO Products VALUES (?, ?, ?, NULL, ?, ?, ?, ?)", 
                               (ans, iid, name, genre, year, price, qty))
            dbconn.commit()
            
            product = Movie(ans, iid, name, genre, year, price, qty)
            products.append(product)
        
        elif ans == 'BOOK':
            iid = input("Item ID: ")
            name = input("Name: ")
            author = input("Author: ")
            price  = float(input("Price dont add '$' or ',': "))
            qty = int(input("Quantity: "))
            db.execute("INSERT INTO Products VALUES (?, ?, ?, ?, NULL, NULL, ?, ?)", 
                               (ans, iid, name, author, price, qty))
            dbconn.commit()
            
            product = Book(ans, iid, name, author, price, qty)
            products.append(product)
        else:
            print("Please enter book or movie")
    
def get_all_products(dbconn) :
    """retrieves all employees data from database"""

    # check if db connection exists
    with dbconn:
        db.execute("SELECT * FROM Products ORDER BY iid")
        return db.fetchall()
                
def initalize_from_file(dbconn, filename, products) :
    """reads csv file and save data into database and list of all products"""
    
    with  open(filename, "r") as data_file : 
        csv_data = csv.reader(data_file)
            
        with closing(dbconn.cursor()) as db :            
            for item in csv_data :
                if (len(item) == 6) :
                    db.execute("INSERT INTO Products VALUES (?, ?, ?, ?, NULL, NULL, ?, ?)", 
                               (item[0], item[1], item[2], item[3], item[4], item[5]))
                    dbconn.commit()
                    product = Book(item[0], item[1], item[2], item[3], item[4], item[5])
                    products.append(product)
                elif (len(item) == 7):
                    db.execute("INSERT INTO Products VALUES (?, ?, ?, NULL, ?, ?, ?, ?)", 
                               (item[0], item[1], item[2], item[3], item[4], item[5], item[6]))
                    dbconn.commit()
                    product = Movie(item[0], item[1], item[2], item[3], item[4], item[5], item[6])
                    products.append(product)
                else :
                    print(f"Invalid file format. Skipping...{item}...")

def list_all_products_menu(dbconn) :
    """lists all employees' information"""
    total = 0
    retval = get_all_products(dbconn)
    print("*** All Products ***")
    print("Catagory Product Name                               Price        Qty           Total")
    print("="*84)
    for item in retval :
        print(f"{item[0].ljust(8)}{item[1].ljust(7)}{item[2].ljust(36)}${item[6]}         {item[7]}        ${round(item[6]*item[7], 2)}")
        total += round(item[6]*item[7], 2)
    print("-"*83)
    print(f"TOTAL                                                                       ${total}")
        
def save_to_file_menu(dbconn, filename) :
    """retrieve data from database and save data to a csv file"""
    
    with dbconn :
        with closing(dbconn.cursor()) as db :
            db.execute("SELECT * FROM Products")

            with  open(filename, "w") as out_csv_file : 
                csv_out = csv.writer(out_csv_file)
                for item in db.fetchall() :
                    csv_out.writerow(item)
                    

In [214]:
FILE_NAME = "productlist.csv"


#In memory database
dbconn = sqlite3.connect(':memory:')

# set a variable up for my cursor so I can write SQL scripts
db = dbconn.cursor()

# create Products table
db.execute("""CREATE TABLE Products (
            catagory TEXT,
            iid TEXT,
            name TEXT,
            author TEXT,
            genre TEXT,
            year INTEGER,
            price REAL,
            qty INTEGER
            )""")

#commits your transactions into the DATABASE
dbconn.commit()

In [215]:
allProducts = []
initalize_from_file(dbconn, FILE_NAME, allProducts)

print("PRODUCT INVENTORY PROGRAM")    
ans = int(0)
while ans != 3:

    ans = int(input("1. Show all products\n2. Add a product\n3. Quit\n\nEnter your choice: "))    
    if ans == 1:
        list_all_products_menu(dbconn)
    elif ans == 2:
        add_product_menu(dbconn, allProducts)
    elif ans == 3:
        print("bye!")
    else:
        print("ERROR: invalid entry try again.")

OperationalError: no such column: NONE

In [208]:
save_to_file_menu(dbconn, FILE_NAME)

In [184]:
dbconn.close()